In [59]:
import pandas as pd
import numpy as np
import math

In [324]:
data = pd.read_csv('pow.csv')
ng = len(data)

In [325]:
team = list(set(list(data['主队']) + list(data['客队'])))
n = len(team)

In [326]:
def diff(homes,aways):
    r = (min(homes,aways))/(max(homes,aways)-1)
    x = 125+475*math.sin(min(1,2*(1-r))*0.4*math.pi)/math.sin(0.4*math.pi)
    return x
def wei(homes,aways):
    return min(1,math.sqrt((max(homes,aways)+max(min(homes,aways),math.floor((max(homes,aways)-1)/2)))/19))

In [327]:
rate0 = dict(zip(team,[1000]*n))
for i in range(1000):
    rate1 = dict(zip(team,[0]*n))
    wei0 = dict(zip(team,[0]*n))
    for j in range(ng):
        home = data.loc[j,'主队']
        away = data.loc[j,'客队']
        homes = data.loc[j,'主分']
        aways = data.loc[j,'客分']
        diffj = diff(homes,aways)
        weij = wei(homes,aways)
        if homes > aways:
            if (rate0[home] - rate0[away] > 600) & (diffj == 600):
                pass
            else:
                rate1[home] += (rate0[away] + diffj)*weij
                rate1[away] += (rate0[home] - diffj)*weij
                wei0[home] += weij
                wei0[away] += weij
        elif homes < aways:
            if (rate0[away] - rate0[home] > 600) & (diffj == 600):
                pass
            else:
                rate1[home] += (rate0[away] - diffj)*weij
                rate1[away] += (rate0[home] + diffj)*weij
                wei0[home] += weij
                wei0[away] += weij
        elif homes == aways:
            rate1[home] += rate0[away]*weij
            rate1[away] += rate0[home]*weij
            wei0[home] += weij
            wei0[away] += weij
    for k in range(n):
        if wei0[team[k]] == 0:
            rate0[team[k]] = -1000
        else:
            rate0[team[k]] = rate1[team[k]]/wei0[team[k]]

In [328]:
rate = pd.DataFrame(list(rate0.items()),columns=['team','rating']).sort_values(by=['rating'],ascending=False)
ranking = rate.reset_index(drop=True)

In [329]:
data1 = data.copy()
data1.columns = ['比赛','客队','客分','主分','主队']
data2 = pd.DataFrame()
for i in range(len(data)):
    data2 = pd.concat([data2,data[i:i+1]],ignore_index=True)
    data2 = pd.concat([data2,data1[i:i+1]],ignore_index=True)

In [330]:
df = pd.DataFrame()
team = list(ranking['team'])
for i in range(n):
    dfi = data2[data2['主队']==team[i]]
    dfi = dfi.reset_index(drop = True)
    change = []
    for j in range(len(dfi)):
        ratej = 0
        weij = 0
        for k in range(len(dfi)):
            if k != j:
                home = dfi.loc[k,'主队']
                away = dfi.loc[k,'客队']
                homes = dfi.loc[k,'主分']
                aways = dfi.loc[k,'客分']
                diffk = diff(homes,aways)
                weik = wei(homes,aways)
                if homes > aways:
                    if (rate0[home] - rate0[away] > 600) & (diffk == 600):
                        pass
                    else:
                        ratej += (rate0[away] + diffk)*weik
                        weij += weik
                elif homes < aways:
                    if (rate0[away] - rate0[home] > 600) & (diffk == 600):
                        pass
                    else:
                        ratej += (rate0[away] - diffk)*weik
                        weij += weik
                elif homes == aways:
                    ratej += rate0[away]*weik
                    weij += weik
        if weij != 0:
            change.append(int(rate0[home]-ratej/weij))
        else:
            change.append(0)
    dfi.insert(1,'change', change)
    df = pd.concat([df,dfi],ignore_index=True)

In [331]:
ranking.to_csv('ranking.csv')
df.to_csv('detail.csv)

,team,rating
0,上海Huwa,2301.843404
1,深圳Zen,2206.020003
2,广州Storm,1986.823669
3,成都Feipanda,1961.501516
4,东莞黑虎帮,1931.990299
5,广州红荔枝,1829.907228
6,长沙1970,1742.064557
7,天津Speed,1714.430636
8,北京MFH,1645.627830
9,深圳Huck,1592.633022
